In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('Assignment1_2020.xlsx', header = [0,1])[:12]
data

Unnamed: 0_level_0                 Single70                            \
               Months Sales\nForecast\n(Units) Production\nCost ($/unit)   
0                 Jan                  25000.0                     450.0   
1                 Feb                  25000.0                     450.0   
2                 Mar                  26000.0                     450.0   
3                 Apr                  26000.0                     475.0   
4                 May                  27000.0                     475.0   
5                 Jun                  27000.0                     475.0   
6                 Jul                  28000.0                     500.0   
7                 Aug                  29000.0                     500.0   
8                 Sep                  30000.0                     500.0   
9                 Oct                  30000.0                     400.0   
10                Nov                  30000.0                     400.0   
11                Dec                  30000.0                     400.0   

                   Double70                            \
   Sales\nForecast\n(Units) Production\nCost ($/unit)   
0                   18500.0                     650.0   
1                   19000.0                     650.0   
2                   20000.0                     650.0   
3                   21000.0                     625.0   
4                   20000.0                     600.0   
5                   19000.0                     590.0   
6                   18000.0                     580.0   
7                   17000.0                     550.0   
8                   19000.0                     550.0   
9                   20000.0                     550.0   
10                  20500.0                     610.0   
11                  21000.0                     640.0   

                   Double80                            
   Sales\nForecast\n(Units) Production\nCost ($/unit)  
0                    9500.0                     810.0  
1                    9500.0                     810.0  
2                    9500.0                     880.0  
3                    7500.0                     880.0  
4                    7500.0                     880.0  
5                    7500.0                     880.0  
6                    6500.0                     880.0  
7                    6500.0                     840.0  
8                    6500.0                     840.0  
9                    6500.0                     840.0  
10                   6500.0                     880.0  
11                   6500.0                     880.0

In [3]:
data['Double70']['Sales\nForecast\n(Units)']

0     18500.0
1     19000.0
2     20000.0
3     21000.0
4     20000.0
5     19000.0
6     18000.0
7     17000.0
8     19000.0
9     20000.0
10    20500.0
11    21000.0
Name: Sales\nForecast\n(Units), dtype: float64

In [4]:
#Constant Workforce
#Overtime Allowed
#Shortages Not Allowed

inv_carry_rate = np.r_[[0.01]*6, [0.013]*6]
prod_times = {'Single70': 1.65, 'Double70': 2.15, 'Double80': 2.68}
reg_workforce = np.r_[[90000]*6, [100000]*6]
overtime_workforce = reg_workforce*0.25
reg_cost = np.r_[[30]*6, [35]*6]
overtime_cost = reg_cost*1.5
I0 =[6500, 12000, 3000]


In [5]:
V = np.c_[data['Single70']['Production\nCost ($/unit)'], data['Double70']['Production\nCost ($/unit)'],data['Double80']['Production\nCost ($/unit)']].T
C = np.multiply(V , inv_carry_rate)
d = np.c_[data['Single70']['Sales\nForecast\n(Units)'], data['Double70']['Sales\nForecast\n(Units)'],data['Double80']['Sales\nForecast\n(Units)']].T
k = np.array(list(prod_times.values()))

## Base Model

In [7]:
import pyomo.environ as pyo
i = 3
t = 12
M_Base = pyo.ConcreteModel()
M_Base.i = range(i)
M_Base.t = range(t)
M_Base.t2 = range(1,t)

M_Base.X = pyo.Var(M_Base.i, M_Base.t, domain = pyo.NonNegativeReals)
M_Base.W = pyo.Var(M_Base.t, domain = pyo.NonNegativeReals)
M_Base.O = pyo.Var(M_Base.t, domain = pyo.NonNegativeReals)
M_Base.I = pyo.Var(M_Base.i, M_Base.t, domain = pyo.NonNegativeReals)

def obj(model):
    return sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) 

M_Base.obj = pyo.Objective(rule = obj, sense= pyo.minimize)

def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t]

M_Base.c1 = pyo.Constraint(M_Base.i, M_Base.t2, rule = c1_rule)
M_Base.c2 = pyo.Constraint(M_Base.t, rule = c2_rule)
M_Base.c3 = pyo.Constraint(M_Base.t, rule = c3_rule)
M_Base.c4 = pyo.Constraint(M_Base.t, rule = c4_rule)
M_Base.c0 = pyo.Constraint(M_Base.i, rule = c0_rule)

In [9]:
opt = pyo.SolverFactory('mosek')
opt.solve(M_Base, tee = True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmp2x0aycxn.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmp2x0aycxn.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 45 (-28) rows, 93 (-4) columns and 159 (-31) elements
Statistics for presolved model


Problem has 45 rows, 93 columns (93 with objective) and 159 elements
There are 27 singletons with objective 
Column breakdown:
66 of type 0.0->inf, 24 of type 0.0->up, 3 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
33 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0

{'Problem': [{'Name': 'unknown', 'Lower bound': 395706458.8, 'Upper bound': 395706458.8, 'Number of objectives': 1, 'Number of constraints': 73, 'Number of variables': 97, 'Number of nonzeros': 93, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 72}}, 'Error rc': 0, 'Time': 0.05585193634033203}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [14]:
def get_results(M):
    X = []
    [X.append(M.X[i,t]()) for i in M.i for t in M.t]
    X =np.reshape(X, (i,t))
    X_df = pd.DataFrame(X.T, columns = list(prod_times.keys()))
    
    I = []
    [I.append(M.I[i,t]()) for i in M.i for t in M.t]
    I =np.reshape(I, (i,t))
    I_df = pd.DataFrame(I.T, columns = list(prod_times.keys()))

    O = []
    [O.append(M.O[t]()) for t in M.t]
    O =np.array(O)
    O_df = pd.DataFrame(O.T, columns = ['Value'])
    
    W = []
    [W.append(M.W[t]()) for t in M.t]
    W =np.array(W)
    W_df = pd.DataFrame(W.T, columns = ['Value'])
    
    return {'X':X_df, 'I':I_df, 'O':O_df, 'W':W_df, 'Objective': M.obj()}
    

In [ ]:
base = get_results(M_Base)
base['Objective']

## Extention I

In [ ]:
bigM = 1000
extra_worker_cost = [bigM]*5 + [38]*3 + [bigM]*4

M_Ext1= pyo.ConcreteModel()
M_Ext1.i = range(i)
M_Ext1.t = range(t)
M_Ext1.t2 = range(1,t)

M_Ext1.E = pyo.Var(M_Ext1.t, domain = pyo.NonNegativeReals)

def obj_ext1(model):
    return sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) + sum(extra_worker_cost[t]*model.E[t] for t in model.t)

In [ ]:
i = 3
t = 12

M_Ext1.X = pyo.Var(M_Ext1.i, M_Ext1.t, domain = pyo.NonNegativeReals)
M_Ext1.W = pyo.Var(M_Ext1.t, domain = pyo.NonNegativeReals)
M_Ext1.O = pyo.Var(M_Ext1.t, domain = pyo.NonNegativeReals)
M_Ext1.I = pyo.Var(M_Ext1.i, M_Ext1.t, domain = pyo.NonNegativeReals)


M_Ext1.obj = pyo.Objective(rule = obj_ext1, sense= pyo.minimize)

def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t] + model.E[t]

M_Ext1.c1 = pyo.Constraint(M_Ext1.i, M_Ext1.t2, rule = c1_rule)
M_Ext1.c2 = pyo.Constraint(M_Ext1.t, rule = c2_rule)
M_Ext1.c3 = pyo.Constraint(M_Ext1.t, rule = c3_rule)
M_Ext1.c4 = pyo.Constraint(M_Ext1.t, rule = c4_rule)
M_Ext1.c0 = pyo.Constraint(M_Ext1.i, rule = c0_rule)

In [ ]:
opt.solve(M_Ext1, tee = True)

In [ ]:
ext1 = get_results(M_Ext1)
ext1['Objective']

In [ ]:
ext1['I']

In [ ]:
ext1['X']

In [ ]:
ext1['W']

In [ ]:
ext1['O']

In [ ]:
E = []
[E.append(M_Ext1.E[t]()) for t in M_Ext1.t]
E =np.array(E)
E_df = pd.DataFrame(E.T, columns = ['Value'])
ext1['E'] = E_df
ext1['E']

## Extention 2

In [ ]:
max_inv = 25000

In [ ]:
M_Ext2 = pyo.ConcreteModel()
M_Ext2.i = range(i)
M_Ext2.t = range(t)
M_Ext2.t2 = range(1,t)

M_Ext2.X = pyo.Var(M_Ext2.i, M_Ext2.t, domain = pyo.NonNegativeReals)
M_Ext2.W = pyo.Var(M_Ext2.t, domain = pyo.NonNegativeReals)
M_Ext2.O = pyo.Var(M_Ext2.t, domain = pyo.NonNegativeReals)
M_Ext2.I = pyo.Var(M_Ext2.i, M_Ext2.t, domain = pyo.NonNegativeReals)

def obj(model):
    return sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) 

M_Ext2.obj = pyo.Objective(rule = obj, sense= pyo.minimize)

def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t]
def extention2_rule(model, t):
    return sum(model.I[i,t] for i in model.i ) <= max_inv

M_Ext2.c1 = pyo.Constraint(M_Ext2.i, M_Ext2.t2, rule = c1_rule)
M_Ext2.c2 = pyo.Constraint(M_Ext2.t, rule = c2_rule)
M_Ext2.c3 = pyo.Constraint(M_Ext2.t, rule = c3_rule)
M_Ext2.c4 = pyo.Constraint(M_Ext2.t, rule = c4_rule)
M_Ext2.c0 = pyo.Constraint(M_Ext2.i, rule = c0_rule)
M_Ext2.c5 = pyo.Constraint(M_Ext2.t, rule = extention2_rule)

In [ ]:
opt.solve(M_Ext2, tee = True)

In [ ]:
ext2 = get_results(M_Ext2)
print(ext2['Objective'])
ext2['Objective'] - ext1['Objective'] 

In [ ]:
ext2['I']

## Extention 3

In [ ]:
bigM = 1000
max_inv = 25000
extra_worker_cost = [bigM]*5 + [38]*3 + [bigM]*4

M_Ext3 = pyo.ConcreteModel()
M_Ext3.i = range(i)
M_Ext3.t = range(t)
M_Ext3.t2 = range(1,t)

M_Ext3.E = pyo.Var(M_Ext3.t, domain = pyo.NonNegativeReals)

def obj_ext1(model):
    return sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) + sum(extra_worker_cost[t]*model.E[t] for t in model.t)

M_Ext3.X = pyo.Var(M_Ext3.i, M_Ext3.t, domain = pyo.NonNegativeReals)
M_Ext3.W = pyo.Var(M_Ext3.t, domain = pyo.NonNegativeReals)
M_Ext3.O = pyo.Var(M_Ext3.t, domain = pyo.NonNegativeReals)
M_Ext3.I = pyo.Var(M_Ext3.i, M_Ext3.t, domain = pyo.NonNegativeReals)


M_Ext3.obj = pyo.Objective(rule = obj_ext1, sense= pyo.minimize)

def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t] + model.E[t]
def extention2_rule(model, t):
    return sum(model.I[i,t] for i in model.i ) <= max_inv

M_Ext3.c1 = pyo.Constraint(M_Ext3.i, M_Ext3.t2, rule = c1_rule)
M_Ext3.c2 = pyo.Constraint(M_Ext3.t, rule = c2_rule)
M_Ext3.c3 = pyo.Constraint(M_Ext3.t, rule = c3_rule)
M_Ext3.c4 = pyo.Constraint(M_Ext3.t, rule = c4_rule)
M_Ext3.c0 = pyo.Constraint(M_Ext3.i, rule = c0_rule)
M_Ext3.c5 = pyo.Constraint(M_Ext3.t, rule = extention2_rule)

In [ ]:
opt.solve(M_Ext3, tee = True)

In [ ]:
ext3 = get_results(M_Ext3)
print(ext3['Objective'])
ext3['Objective'] - ext2['Objective'] 

In [ ]:
E = []
[E.append(M_Ext3.E[t]()) for t in M_Ext3.t]
E =np.array(E)
E_df = pd.DataFrame(E.T, columns = ['Value'])
ext3['E'] = E_df
ext3['E']

## Extention 4

In [ ]:
inv_carry_rate2 = [0.01]*6 + [0.02]*6
C2 = np.multiply(V , inv_carry_rate2)

In [ ]:
M_Ext4 = pyo.ConcreteModel()
M_Ext4.i = range(i)
M_Ext4.t = range(t)
M_Ext4.t2 = range(1,t)

def obj(model):
    return sum(C2[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t)

M_Ext4.X = pyo.Var(M_Ext4.i, M_Ext4.t, domain = pyo.NonNegativeReals)
M_Ext4.W = pyo.Var(M_Ext4.t, domain = pyo.NonNegativeReals)
M_Ext4.O = pyo.Var(M_Ext4.t, domain = pyo.NonNegativeReals)
M_Ext4.I = pyo.Var(M_Ext4.i, M_Ext4.t, domain = pyo.NonNegativeReals)


M_Ext4.obj = pyo.Objective(rule = obj, sense= pyo.minimize)

def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t]

M_Ext4.c1 = pyo.Constraint(M_Ext4.i, M_Ext4.t2, rule = c1_rule)
M_Ext4.c2 = pyo.Constraint(M_Ext4.t, rule = c2_rule)
M_Ext4.c3 = pyo.Constraint(M_Ext4.t, rule = c3_rule)
M_Ext4.c4 = pyo.Constraint(M_Ext4.t, rule = c4_rule)
M_Ext4.c0 = pyo.Constraint(M_Ext4.i, rule = c0_rule)


In [ ]:
opt.solve(M_Ext4, tee = True)

In [ ]:
ext4 = get_results(M_Ext4)
print(ext4['Objective'])
ext4['Objective'] - base['Objective']

## Extension 5

In [23]:
def Ext5(k_rate):
    M_Ext5_1 = pyo.ConcreteModel()
    M_Ext5_1.i = range(i)
    M_Ext5_1.t = range(t)
    M_Ext5_1.t2 = range(1,t)

    M_Ext5_1.X = pyo.Var(M_Ext5_1.i, M_Ext5_1.t, domain = pyo.NonNegativeReals)
    M_Ext5_1.W = pyo.Var(M_Ext5_1.t, domain = pyo.NonNegativeReals)
    M_Ext5_1.O = pyo.Var(M_Ext5_1.t, domain = pyo.NonNegativeReals)
    M_Ext5_1.I = pyo.Var(M_Ext5_1.i, M_Ext5_1.t, domain = pyo.NonNegativeReals)

    def obj(model):
        return sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) 

    M_Ext5_1.obj = pyo.Objective(rule = obj, sense= pyo.minimize)

    def c0_rule(model, i):
        return model.X[i,0] + I0[i] - d[i,0] -  model.I[i,0] == 0
    def c1_rule(model, i, t2):
        return model.X[i,t2] + model.I[i,t2-1] - d[i,t2] - model.I[i,t2] == 0
    def c2_rule(model, t):
        return model.W[t] <= reg_workforce[t]
    def c3_rule(model, t):
        return model.O[t] <= overtime_workforce[t]
    def c4_rule(model, t):
        return sum(model.X[i,t]*k[i]*k_rate[i] for i in model.i) <= model.O[t] + model.W[t]
    def extention2_rule(model, t):
        return sum(model.I[i,t] for i in model.i ) <= max_inv

    M_Ext5_1.c1 = pyo.Constraint(M_Ext5_1.i, M_Ext5_1.t2, rule = c1_rule)
    M_Ext5_1.c2 = pyo.Constraint(M_Ext5_1.t, rule = c2_rule)
    M_Ext5_1.c3 = pyo.Constraint(M_Ext5_1.t, rule = c3_rule)
    M_Ext5_1.c4 = pyo.Constraint(M_Ext5_1.t, rule = c4_rule)
    M_Ext5_1.c0 = pyo.Constraint(M_Ext5_1.i, rule = c0_rule)
    M_Ext5_1.c5 = pyo.Constraint(M_Ext5_1.t, rule = extention2_rule)
    
    opt.solve(M_Ext5_1, tee = False)
    ext5_1 = get_results(M_Ext5_1)
    
    return ext5_1

In [24]:
from itertools import product
ext5_res ={}
for p in product((1.1, 0.9), repeat=3):
    ext5_res[p] = Ext5(p)['Objective']
    

In [25]:
ext5_res

{(1.1, 1.1, 1.1): 403033533.004255,
 (1.1, 1.1, 0.9): 400063669.36012,
 (1.1, 0.9, 1.1): 397580235.86978,
 (1.1, 0.9, 0.9): 395179299.19487,
 (0.9, 1.1, 1.1): 397061021.549275,
 (0.9, 1.1, 0.9): 394735107.7891,
 (0.9, 0.9, 1.1): 392612661.2126,
 (0.9, 0.9, 0.9): 390512284.75321996}

## Extention 6

In [18]:
bigM = 100000000
M_Ext6 = pyo.ConcreteModel()
M_Ext6.i = range(i)
M_Ext6.t = range(t)
M_Ext6.t2 = range(1,t)
max_inv = 25000
#update demand
d2 = d.copy()
d2[2,3] = 17500
d2[2,4] =  17500
d2[2,5] =  17500
#Shortage Variable
M_Ext6.S = pyo.Var(M_Ext6.i, M_Ext6.t, domain = pyo.NonNegativeReals)

In [19]:
d2

array([[25000., 25000., 26000., 26000., 27000., 27000., 28000., 29000.,
        30000., 30000., 30000., 30000.],
       [18500., 19000., 20000., 21000., 20000., 19000., 18000., 17000.,
        19000., 20000., 20500., 21000.],
       [ 9500.,  9500.,  9500., 17500., 17500., 17500.,  6500.,  6500.,
         6500.,  6500.,  6500.,  6500.]])

In [20]:
M_Ext6.X = pyo.Var(M_Ext6.i, M_Ext6.t, domain = pyo.NonNegativeReals)
M_Ext6.W = pyo.Var(M_Ext6.t, domain = pyo.NonNegativeReals)
M_Ext6.O = pyo.Var(M_Ext6.t, domain = pyo.NonNegativeReals)
M_Ext6.I = pyo.Var(M_Ext6.i, M_Ext6.t, domain = pyo.NonNegativeReals)

#Include bigM*S to ensure shortage is the latest choice of the model
def obj_ext6(model):
    return sum(model.S[i,t]*bigM for i in model.i for t in model.t) + sum(C[i,t]*model.I[i,t] for i in model.i for t in model.t) + sum(V[i,t]*model.X[i,t] for i in model.i for t in model.t) + sum(reg_cost[t]*model.W[t] for t in model.t) + sum(overtime_cost[t]*model.O[t] for t in model.t) 

M_Ext6.obj = pyo.Objective(rule = obj_ext6, sense= pyo.minimize) #obj_ext6, sense= pyo.minimize)

#Update demand constraints (c0, c1) to include shortages
def c0_rule(model, i):
    return model.X[i,0] + I0[i] - d2[i,0] -  model.I[i,0] + model.S[i,0] == 0
def c1_rule(model, i, t2):
    return model.X[i,t2] + model.I[i,t2-1] - d2[i,t2] - model.I[i,t2] + model.S[i,t2] == 0
def c2_rule(model, t):
    return model.W[t] <= reg_workforce[t]
def c3_rule(model, t):
    return model.O[t] <= overtime_workforce[t]
def c4_rule(model, t):
    return sum(model.X[i,t]*k[i] for i in model.i) <= model.O[t] + model.W[t]
def extention2_rule(model, t):
    return sum(model.I[i,t] for i in model.i ) <= max_inv

M_Ext6.c1 = pyo.Constraint(M_Ext6.i, M_Ext6.t2, rule = c1_rule)
M_Ext6.c2 = pyo.Constraint(M_Ext6.t, rule = c2_rule)
M_Ext6.c3 = pyo.Constraint(M_Ext6.t, rule = c3_rule)
M_Ext6.c4 = pyo.Constraint(M_Ext6.t, rule = c4_rule)
M_Ext6.c0 = pyo.Constraint(M_Ext6.i, rule = c0_rule)
M_Ext6.c5 = pyo.Constraint(M_Ext6.t, rule = extention2_rule)


In [21]:
opt.solve(M_Ext6, tee = True)

Welcome to the CBC MILP Solver 
Version: Trunk (unstable) 
Build Date: Feb 24 2019 

command line - C:\Program Files\CBC 2.10.0\bin\cbc.exe -printingOptions all -import C:\Users\mehmet\AppData\Local\Temp\tmplajyoh66.pyomo.lp -stat=1 -solve -solu C:\Users\mehmet\AppData\Local\Temp\tmplajyoh66.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 60 (-25) rows, 132 (-1) columns and 237 (-25) elements
Statistics for presolved model


Problem has 60 rows, 132 columns (132 with objective) and 237 elements
There are 60 singletons with objective 
Column breakdown:
108 of type 0.0->inf, 24 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
36 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 12 of type L 0.0, 0 of type L 1.0, 
12 of type L other, 0 of type Range 0.0->1

{'Problem': [{'Name': 'unknown', 'Lower bound': 281767268900.0, 'Upper bound': 281767268900.0, 'Number of objectives': 1, 'Number of constraints': 85, 'Number of variables': 133, 'Number of nonzeros': 132, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 82}}, 'Error rc': 0, 'Time': 0.0578458309173584}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
ext6 = get_results(M_Ext6)
ext6['Objective']

2531767192164.099

In [22]:
S = []
[S.append(M_Ext6.S[i,t]()) for i in M_Ext6.i for t in M_Ext6.t]
S =np.reshape(S, (i,t))
S_df = pd.DataFrame(S.T, columns = list(prod_times.keys()))
ext6['S'] = S_df
ext6['S']

,Single70,Double70,Double80
0,0.0,0.0,0.0000
1,0.0,0.0,0.0000
2,0.0,0.0,0.0000
3,0.0,0.0,0.0000
4,0.0,0.0,0.0000
5,0.0,0.0,2813.4328
6,0.0,0.0,0.0000
7,0.0,0.0,0.0000
8,0.0,0.0,0.0000
9,0.0,0.0,0.0000


In [ ]:
ext6['I']

In [ ]:
ext6['X']

In [ ]:
M_Base.display()

In [ ]:
M_Base.pprint()